## Парсинг данных сайта 3dsky.org

### Получение данных с сайта
Получение данных выполняется в два прохода. Сначала мы берём первые страницы топа и формируем список моделей с основными данными: номер по порядку, дата парсинга, наименование, ссылка. На втором этапе мы проходим по этому списку, заходя на страницу каждой модели, и получаем детализированную информацию о ней. В связи с тем, что иногда случаются ошибки получения данных (из-за неустойчивости сети), для каждой модели предпринимаются несколько попыток парсинга.

In [7]:
# Парсинг топа продаж 3dsky
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import time

# Настройка Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Запуск без интерфейса (опционально)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_service = Service("C:\\Users\\mi_al\\Desktop\\chromedriver-win64\\chromedriver.exe")  # Замените на путь к вашему ChromeDriver

# Инициализация констант и переменных
result = []                                                         # итоговый список
maxpage = 2                                                         # сколько страниц топа грузим
basic_url = "https://3dsky.org/3dmodels?order=sell_rating&page="    # адрес (без номера страницы)
pause_time = 5                                                      # время задержки на загрузку в сек
page_size = 60                                                      # число моделей на странице
attempts = 5                                                        # число попыток парсинга каждой модели
basic_width = 4                                                     # количество параметров модели из топа
full_width = 14                                                     # полное число параметров модели
url_position = 3                                                    # на каком месте находится параметр url

# Формируем список моделей
for page in range(maxpage):
    # Инициализация браузера
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    
    url = basic_url + str(page + 1)
    driver.get(url)

    # Небольшая пауза для загрузки контента
    time.sleep(pause_time)

    # Парсинг данных
    try:
        # Получаем список моделей с заданной страницы топа
        models = driver.find_elements(By.CSS_SELECTOR, "div.model-title")

        for n, model in enumerate(models, 1):
            # Извлечение данных
            curdate = str(datetime.now().date())
            title = model.find_element(By.TAG_NAME, "a").get_attribute("title") if model.find_elements(By.TAG_NAME, "a") else "No title"
            link = model.find_element(By.TAG_NAME, "a").get_attribute("href") if model.find_elements(By.TAG_NAME, "a") else "No link"
            #print(f"Название: {title}, Ссылка: {link}")
            
            # Формируем заготовку строки: номер строки, текущая дата, название модели, ссылка              
            line_num = page*page_size + n
            result.append([line_num, curdate, title, link])

    finally:
        # Закрытие браузера
        driver.quit()

# В цикле по списку получаем подробную информацию по каждой модели
for model in result:
    for attempt in range(attempts):
    
        # Инициализация браузера
        driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
        
        # Открытие страницы конкретной модели
        url = model[url_position]
        driver.get(url)
        
        # Небольшая пауза для загрузки контента
        time.sleep(pause_time)
        
        # Парсинг данных
        try:
            category = driver.find_elements(By.CSS_SELECTOR, "body > app-root > app-model > app-base-wide > main > section.container.main-base-container > div.row.white-background > div.col-md-12.model-page-top.ng-tns-c53-0 > div.favourite-and-category.ng-tns-c53-0.ng-star-inserted > div:nth-child(1) > div.category.ng-tns-c53-0 > a > span")
            subcategory = driver.find_elements(By.CSS_SELECTOR, "body > app-root > app-model > app-base-wide > main > section.container.main-base-container > div.row.white-background > div.col-md-12.model-page-top.ng-tns-c53-0 > div.favourite-and-category.ng-tns-c53-0.ng-star-inserted > div:nth-child(1) > div.subcategory.ng-tns-c53-0 > a > span")
            platform = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div.model-info-block.ng-tns-c53-0 > table > tbody > tr:nth-child(1) > td:nth-child(2)")
            renders = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div.model-info-block.ng-tns-c53-0 > table > tbody > tr:nth-child(2) > td:nth-child(2) > div")
            published = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div.model-info-block.ng-tns-c53-0 > div.publication-date.ng-tns-c53-0.ng-star-inserted")
            username = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div:nth-child(1) > div > div > div.author-of-model.ng-tns-c53-0.no-button.ng-star-inserted > a > div > div > div.model-user-name.ng-tns-c53-0")
            followers = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div:nth-child(1) > div > div > div.author-of-model.ng-tns-c53-0.no-button.ng-star-inserted > a > div > div > div.model-subscribe-count.ng-tns-c53-0.ng-star-inserted")
            selected = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div:nth-child(1) > div > div > div:nth-child(3) > div.price-block.ng-tns-c53-0.ng-star-inserted > div.bookmarks-block-wrapper-mobile.ng-tns-c53-0.ng-star-inserted > div > div")
            
            # Добавляем полученные данные в строку: категория, подкатегория, платформа, рендер
            if category:
                model.append(category[0].text)
            if subcategory:
                model.append(subcategory[0].text)
            if platform:
                model.append(platform[0].text)
            
            if renders:
                render_text = renders[0].text
            if ('Corona' in render_text) and ('V-Ray' in render_text):
                model.extend(['Corona','V-Ray','None'])
            elif ('Corona' in render_text):
                model.extend(['Corona','None','None'])
            elif ('V-Ray' in render_text):
                model.extend(['None','V-Ray','None'])
            elif ('Standard' in render_text):
                model.extend(['None','None','Standard'])
            else:
                model.extend(['None','None','None'])
            
            if published:
                pub_date = " ".join(str(published[0].text).split(" ")[1:])
                date_object = datetime.strptime(pub_date, "%d %B %Y")
                formatted_date = date_object.strftime("%Y-%m-%d")
                model.append(formatted_date)
            if username:
                model.append(username[0].text)
            if followers:
                model.append(int(followers[0].text.split(" ")[0]))
            if selected:
                selected_text = driver.execute_script("return arguments[0].innerText;", selected[0])
                model.append(int(selected_text.strip()))

        finally:
            # Закрытие браузера
            driver.quit()
            
        # Проверяем, все ли параметры считались, и если нет, то пробуем ещё несколько раз
        if len(model) == full_width:
            print(f'Success: {model}, attempt: {attempt+1}')
            break
        else:
            if attempt+1 == attempts:
                print(f'Failed: {model}, attempt: {attempt+1}')
            else:
                print(f'Pending: {model}, attempt: {attempt+1}')
                if len(model) > basic_width:
                    del model[basic_width:]


Success: [1, '2025-10-13', 'Amaranth of two types (green and burgundy) in a Gordon Baldwin vase', 'https://3dsky.org/3dmodels/show/amarant-dvukh-vidov-zelenyi-i-bordovyi-v-vaze-gordon-baldwin', 'Plants', 'Bouquet', '3dsMax 2019 + fbx', 'Corona', 'None', 'None', '2025-09-21', 'moloko_w', 224, 336], attempt: 1
Success: [2, '2025-10-13', 'Boheme Uno Mixer Set', 'https://3dsky.org/3dmodels/show/nabor_smesitelei_boheme_uno_2', 'Bathroom', 'Faucet', '3dsMax 2018 + obj', 'Corona', 'None', 'None', '2025-08-05', 'ArturZH', 16, 90], attempt: 1
Success: [3, '2025-10-13', 'Wardrobe Novamobili Perry 2', 'https://3dsky.org/3dmodels/show/garderob_novamobili_perry_2', 'Furniture', 'Wardrobe & Display cabinets', '3dsMax 2013 + obj', 'Corona', 'V-Ray', 'None', '2025-07-28', 'restereo', 171, 303], attempt: 1
Success: [4, '2025-10-13', "Children's room wardrobe with workspace 4", 'https://3dsky.org/3dmodels/show/shkaf-dlia-detskoi-komnaty-s-rabochim-mestom-4', 'Childroom', 'Wardrobe', '3dsMax 2015 + obj',

### Запись результатов парсинга в файл

In [8]:
# Добавляем результат в конец файла
import pandas as pd
file_path = "output.xlsx"

# Чтение данных
existing_data = pd.read_excel(file_path)

# Преобразование текущих данных DataFrame
new_data_df = pd.DataFrame(result[0:], columns=['N','Curdate','Title','Link','Category','Subcategory','Platform','Corona','V-Ray','Standard','Pubdate','Username','Followers','Selected'])

# Конкатенация и запись результата в Excel
updated_data = pd.concat([existing_data, new_data_df], ignore_index=True)
updated_data.to_excel(file_path, index=False)

# Выводим результат для контроля
print(f"Данные успешно сохранены в {file_path}")


Данные успешно сохранены в output.xlsx


In [6]:
# Песочница для апробации парсинга конкретного элемента
# Парсинг топа продаж 3dsky
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import time


# Настройка Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Запуск без интерфейса (опционально)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_service = Service("C:\\Users\\mi_al\\Desktop\\chromedriver-win64\\chromedriver.exe")  # Замените на путь к вашему ChromeDriver

# Инициализация браузера
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Открытие сайта
url = "https://3dsky.org/3dmodels/show/kerns_tumba_divan_ru"
driver.get(url)

# Небольшая пауза для загрузки контента
time.sleep(5)  # Настройте задержку в зависимости от скорости интернета

# Парсинг данных
try:
    # Пример: Найти все элементы с моделями (настройте селектор под сайт)
    models = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div:nth-child(1) > div > div > div.author-of-model.ng-tns-c56-0.no-button.ng-star-inserted > a > div > div > div.model-subscribe-count.ng-tns-c56-0.ng-star-inserted")
    print(models[0].text)
    #print(models)
    #published = str(models[0].text)

finally:
    # Закрытие браузера
    driver.quit()

899 followers


## Анализ текущего топа
В этом разделе анализируется только топ одного дня, без чтения данных из файла. Для анализа всего топа — см. 3ddd_analysis.ipynb.

In [14]:
# Анализ текущего топа (без чтения из файла)
import pandas as pd
import plotly.express as px

result_vis = pd.DataFrame(result, columns=['N','Curdate','Title','Link','Category','Subcategory','Platform','Corona','V-Ray','Standard','Pubdate','Username','Followers','Selected'])
result_vis['Pubdate'] = pd.to_datetime(result_vis['Pubdate'])
#result_vis.drop('N', axis=1, inplace=True)

current_date = datetime.now()
result_vis['Duration'] = (current_date - result_vis['Pubdate']).dt.days

display(result_vis.head())

,N,Curdate,Title,Link,Category,Subcategory,Platform,Corona,V-Ray,Standard,Pubdate,Username,Followers,Selected,Duration
0,1,2025-01-31,Benuta Leon Cream Wool Rug,https://3dsky.org/3dmodels/show/benuta_leon_cr...,Decoration,Carpets,3dsMax 2015 + obj,Corona,None,None,2024-11-03,3Dmitruk,352,443.0,89
1,2,2025-01-31,A set of clothes and accessories to fill a war...,https://3dsky.org/3dmodels/show/a_set_of_cloth...,Decoration,Clothes,3dsMax 2015 + fbx,Corona,V-Ray,None,2024-11-23,sofiakholina,843,234.0,69
2,3,2025-01-31,Cairns pedestal Divan.ru,https://3dsky.org/3dmodels/show/kerns_tumba_di...,Furniture,Sideboard & Chest of drawer,3dsMax 2016 + obj,Corona,V-Ray,None,2024-11-05,niga2323,811,181.0,87
3,4,2025-01-31,Boheme Uno collection,https://3dsky.org/3dmodels/show/boheme_uno_col...,Bathroom,Faucet,3dsMax 2015 + obj,Corona,V-Ray,None,2025-01-07,sarabale,3,36.0,24
4,5,2025-01-31,curtains with piping,https://3dsky.org/3dmodels/show/shtory_s_kantom,Decoration,Curtain,3dsMax 2018 + fbx,Corona,None,None,2025-01-14,бойко75,21,111.0,17


In [15]:
result_grouped_cat = result_vis.groupby(by=['Category'], as_index=False).size()
result_grouped_cat.columns = ['Category', 'Count']
result_grouped_sort = result_grouped_cat.sort_values(by='Count', ascending=False) # type: ignore

fig1 = px.bar(result_grouped_sort, x='Category', y='Count', title='Top 120 Count by Category')
fig1.show()

In [16]:
result_grouped_subcat = result_vis.groupby(by=['Category', 'Subcategory'], as_index=False).size()
result_grouped_subcat.columns = ['Category', 'Subcategory', 'Count']
result_grouped_subsort = result_grouped_subcat.sort_values(by='Count', ascending=False) # type: ignore

fig2 = px.bar(result_grouped_subsort, x='Subcategory', y='Count', color = 'Category', title='Top 120 Count by Subcategory')
fig2.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [17]:
fig3 = px.scatter(result_vis, x='Selected', y='Followers', color='N', log_x=True, log_y=True, hover_data=['Title','Selected','Followers'], title='Top 120 by Selected and Followers')
fig3.show()

In [19]:
result_vis['Dur_Shrink'] = result_vis['Duration'].apply(lambda x: x if x<500 else 500)
#display(result_vis)

fig4 = px.histogram(result_vis, x='Dur_Shrink', nbins=50, title='Top 120 Count by Duration')
fig4.show()

In [20]:
result_vis['Ranking'] = 121 - result_vis['N']

result_grouped_cat_ranking = result_vis.groupby(by=['Category'], as_index=False)['Ranking'].sum()
result_grouped_cat_ranking.columns = ['Category', 'Total_Ranking']
result_grouped_sort_ranking = result_grouped_cat_ranking.sort_values(by='Total_Ranking', ascending=False) # type: ignore

fig5 = px.bar(result_grouped_sort_ranking, x='Category', y='Total_Ranking', title='Top 120 total Rank by Category')
fig5.show()

In [21]:
fig6 = px.box(result_vis, x='Category', y='Ranking', color='Category', title='Top 120 Rank distribution by Category')
fig6.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [22]:
fig7 = px.box(result_vis, x='Category', y='Dur_Shrink', color='Category', title='Top 120 Duration distribution by Category')
fig7.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [23]:
fig8 = px.scatter(result_vis, x='Dur_Shrink', y='Ranking', color='Category', size='Followers', hover_data=['N','Title','Username','Followers'], title='Top 120 by Duration and Ranking')
fig8.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [24]:
fig1.write_html("Top 120 Count by Category.html")
fig2.write_html("Top 120 Count by Subcategory.html")
fig3.write_html("Top 120 by Selected and Followers.html")
fig4.write_html("Top 120 Count by Duration.html")
fig5.write_html("Top 120 total Rank by Category.html")
fig6.write_html("Top 120 Rank distribution by Category.html")
fig7.write_html("Top 120 Duration distribution by Category.html")
fig8.write_html("Top 120 by Duration and Ranking.html")

In [25]:
fig9 = px.box(result_vis, x='Subcategory', y='Ranking', color='Subcategory', title='Top 120 Rank distribution by Subcategory')
fig9.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [27]:
fig10 = px.box(result_vis, x='Subcategory', y='Dur_Shrink', color='Subcategory', title='Top 120 Duration distribution by Subcategory')
fig10.show()

c:\Users\mi_al\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [45]:
pd.set_option('display.max_colwidth', None)

result_duration = result_vis.sort_values(by='Duration', ascending=False) # type: ignore
display(result_duration[['Link', 'Duration']].reset_index(drop=True).head(10))



,Link,Duration
0,https://3dsky.org/3dmodels/show/floorgen_tools_1_5_4,2119
1,https://3dsky.org/3dmodels/show/laundry_set_poliform_fitted_asko,1987
2,https://3dsky.org/3dmodels/show/dukhovki_i_mikrovolnovki_bosch,1642
3,https://3dsky.org/3dmodels/show/tennisnyi_kort_2,1456
4,https://3dsky.org/3dmodels/show/laundry_room_0005,1227
5,https://3dsky.org/3dmodels/show/washing_machine_and_dryer_samsung,967
6,https://3dsky.org/3dmodels/show/nabor_potolkov_armstrong_i_griliato_1,897
7,https://3dsky.org/3dmodels/show/oborudovanie_dlia_kafe_set_4_1,695
8,https://3dsky.org/3dmodels/show/topol_deltovidnyi_poplar_populus_deltoides_9,476
9,https://3dsky.org/3dmodels/show/moiki_i_smesiteli_omoikiri,474
